## Source: Wikipedia
## Retriever: Wikipedia Search
## Model: Roberta Large BoolQ

In [1]:
import torch
from sklearn.metrics import classification_report, roc_auc_score

import sys
sys.path.append("../../") # use utils

import utils
import importlib
importlib.reload(utils)

from utils import calc_auc
from progressbar import progressbar as pb

In [3]:
from transformers import AutoTokenizer, RobertaForSequenceClassification

model_name = "apugachev/roberta-large-boolq-finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)
model.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print("OK")

OK


In [4]:
def predict(question, passage, device=device):
    sequence = tokenizer.encode_plus(
        question, 
        passage, 
        return_tensors="pt",
        max_length=tokenizer.model_max_length,
        truncation=True
    )['input_ids']

    logits = model(sequence.to(device))[0]
    probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]
    proba_yes = probabilities[1]
    
    return proba_yes

In [2]:
import pandas as pd
init_data = pd.read_csv("../../../data/data_to_process.csv")
print(len(init_data))
init_data.head(3)

113


,data_source,query_id,description,query,label
0,2019,1,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0
1,2019,3,Can acupuncture be effective for people with e...,acupuncture epilepsy,0.0
2,2019,5,Can acupuncture prevent migraines?,acupuncture migraine,1.0


## Keywords

In [3]:
raw_df = pd.read_csv("../../../data/wikipedia_search_wiki/wikipedia_articles_keywords.csv")

df = (
    raw_df
    .merge(init_data, on=["data_source", "query_id", "label"])
)
print(len(df))
df.head(1)

513


,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description,query
0,cranberries urinary tract infections,Urinary tract infection,Some research suggests that cranberry (juice o...,1.0,"['infection', 'urinary', 'tract']",3.0,['cranberry'],['cranberry'],1.0,77.0,1,2019,0.0,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections


In [7]:
results = []

for _, row in pb(df.iterrows(), max_value=len(df)):
    results.append(predict(row.description, row.paragraph))

100% (513 of 513) |######################| Elapsed Time: 0:00:16 Time:  0:00:16


In [8]:
df['prediction'] = results
df.head(1)

,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description,query,prediction
0,cranberries urinary tract infections,Urinary tract infection,Some research suggests that cranberry (juice o...,1.0,"['infection', 'urinary', 'tract']",3.0,['cranberry'],['cranberry'],1.0,77.0,1,2019,0.0,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.940501


In [4]:
QUERY_SAVE_PRED_FILE = "predictions/roberta_large_boolq_wikipedia_search_wiki_preds_keywords.csv"

df.to_csv(QUERY_SAVE_PRED_FILE, index=0)

## Calc Metrics

In [6]:
init_data = pd.read_csv("../../../data/data_to_process.csv")
print(len(init_data))
init_data.head(3)

113


,data_source,query_id,description,query,label
0,2019,1,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0
1,2019,3,Can acupuncture be effective for people with e...,acupuncture epilepsy,0.0
2,2019,5,Can acupuncture prevent migraines?,acupuncture migraine,1.0


In [7]:
df = pd.read_csv(QUERY_SAVE_PRED_FILE)
print(len(df))

513


In [8]:
agg_types = ["avg", "top1", "norm_linear", "norm_log"]
data_source_types = sorted(df.data_source.unique().tolist()) + ["all"]
data_source_types

['2019', '2021', 'health_belief', 'misbelief', 'all']

In [9]:
df_filled = (
    df
    .merge(init_data, how='outer', on=["query_id", "data_source", "label"])
    .fillna({"prediction": 0.5})
)
print(len(df_filled))
df_filled.tail(2)

539


,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description_x,query_x,prediction,description_y,query_y
537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12_h,health_belief,1.0,NaN,NaN,0.5,Does cialis treat enlarged prostrate?,cialis treats enlarged prostrate
538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13_h,health_belief,1.0,NaN,NaN,0.5,Does diet cause bad breathe?,diet causes bad breathe


In [10]:
assert 113 == len(df_filled.drop_duplicates(["query_id", "data_source"]))

In [11]:
metrics = {item: [] for item in data_source_types}

for data_source in data_source_types:
    for agg_type in agg_types:
        if data_source == "all":
            metrics[data_source].append(calc_auc(df_filled, agg_type))
        else:
            df_cut = df_filled.query(f"data_source == '{data_source}'")
            metrics[data_source].append(calc_auc(df_cut, agg_type))

metrics_df = pd.DataFrame(metrics, index=agg_types).round(4)
metrics_df

,2019,2021,health_belief,misbelief,all
avg,0.8062,0.8224,0.7917,0.5804,0.7678
top1,0.8616,0.8384,0.7500,0.6875,0.8060
norm_linear,0.8097,0.8480,0.7500,0.5804,0.7766
norm_log,0.8097,0.8336,0.7500,0.5804,0.7741


## Question

In [12]:
raw_df = pd.read_csv("../../../data/wikipedia_search_wiki/wikipedia_articles_question.csv")

df = (
    raw_df
    .merge(init_data, on=["data_source", "query_id", "label"])
)
print(len(df))
df.head(1)

298


,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description,query
0,Can cranberries prevent urinary tract infections?,Urinary tract infection,Some research suggests that cranberry (juice o...,1.0,"['infection', 'urinary', 'tract']",3.0,"['cranberry', 'prevent', 'can']","['cranberry', 'can']",2.0,77.0,1,2019,0.0,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections


In [23]:
results = []

for _, row in pb(df.iterrows(), max_value=len(df)):
    results.append(predict(row.description, row.paragraph))

100% (298 of 298) |######################| Elapsed Time: 0:00:11 Time:  0:00:11


In [24]:
df['prediction'] = results
df.head(1)

,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description,query,prediction
0,Can cranberries prevent urinary tract infections?,Urinary tract infection,Some research suggests that cranberry (juice o...,1.0,"['infection', 'urinary', 'tract']",3.0,"['cranberry', 'prevent', 'can']","['cranberry', 'can']",2.0,77.0,1,2019,0.0,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.940501


In [13]:
DESCRIPTION_SAVE_PRED_FILE = "predictions/roberta_large_boolq_wikipedia_search_wiki_preds_question.csv"

df.to_csv(DESCRIPTION_SAVE_PRED_FILE, index=0)

## Calc Metrics

In [14]:
df = pd.read_csv(DESCRIPTION_SAVE_PRED_FILE)
print(len(df))

298


In [15]:
df_filled = (
    df
    .merge(init_data, how='outer', on=["query_id", "data_source", "label"])
    .fillna({"prediction": 0.5})
)
print(len(df_filled))
df_filled.tail(2)

354


,query_processed,article_title,paragraph,serp_position,title_common_tokens,num_title_common_tokens,tokens_to_find,paragraph_common_tokens,num_paragraph_common_tokens,paragraph_number_in_article,query_id,data_source,label,description_x,query_x,prediction,description_y,query_y
352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13_h,health_belief,1.0,NaN,NaN,0.5,Does diet cause bad breathe?,diet causes bad breathe
353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14_h,health_belief,1.0,NaN,NaN,0.5,Does listeria cause miscarriage?,listeria causes miscarriage


In [28]:
assert 113 == len(df_filled.drop_duplicates(["query_id", "data_source"]))

In [17]:
metrics = {item: [] for item in data_source_types}

for data_source in data_source_types:
    for agg_type in agg_types:
        if data_source == "all":
            metrics[data_source].append(calc_auc(df_filled, agg_type))
        else:
            try:
                df_cut = df_filled.query(f"data_source == '{data_source}'")
                metrics[data_source].append(calc_auc(df_cut, agg_type))
            except ValueError:
                print(f"Can't calc auc for {data_source} {agg_type}")
            
for key, value in metrics.items():
    if not value:
        metrics[key] = [None] * 4

metrics_df = pd.DataFrame(metrics, index=agg_types).round(4)
metrics_df

,2019,2021,health_belief,misbelief,all
avg,0.8062,0.6992,0.4583,0.4286,0.6799
top1,0.7543,0.7328,0.4583,0.3214,0.6455
norm_linear,0.7716,0.6752,0.4583,0.4286,0.6496
norm_log,0.7855,0.7296,0.4583,0.3214,0.6758
